Cargar Data 

In [1]:
import bootcampviztools as bt
import hashlib
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import modulo_toolbox as mtb
import requests
import seaborn as sns
import time
import toolbox_ML as tb
import warnings

warnings.filterwarnings('ignore')

In [2]:
df_DC = pd.read_csv('./data/heroes_dc_extended')

In [3]:
df_DC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       73 non-null     int64 
 1   nombre_completo  61 non-null     object
 2   nombre           73 non-null     object
 3   universo         73 non-null     object
 4   url              73 non-null     object
 5   Intelligence     73 non-null     int64 
 6   Strength         73 non-null     int64 
 7   Speed            73 non-null     int64 
 8   Durability       73 non-null     int64 
 9   Power            73 non-null     int64 
 10  Combat           73 non-null     int64 
 11  Tier             73 non-null     int64 
 12  Tier2            9 non-null      object
 13  heroe            73 non-null     object
dtypes: int64(8), object(6)
memory usage: 8.1+ KB


In [4]:
# Extraer nombres de los héroes
heroes = df_DC['heroe'].tolist()

In [5]:
# Generar combinaciones ficticias de enfrentamientos
enfrentamientos = list(itertools.combinations(heroes, 2))

In [6]:
# Crear el dataframe de enfrentamientos
enfrentamientos_df = pd.DataFrame(enfrentamientos, columns=['Heroe_1', 'Heroe_2'])


In [7]:
# Guardar el dataframe de enfrentamientos en un archivo CSV
output_file_path = './data/enfrentamientos_heroes_dc.csv'
enfrentamientos_df.to_csv(output_file_path, index=False)

output_file_path


'./data/enfrentamientos_heroes_dc.csv'

BATALLAS Y RESULTADO

In [8]:
battle_DC = pd.read_csv('./data/enfrentamientos_heroes_dc.csv')

In [9]:
battle_DC

,Heroe_1,Heroe_2
0,Aquaman,Ares
1,Aquaman,Atom Smasher
2,Aquaman,Batman
3,Aquaman,Black Adam
4,Aquaman,Black Canary
...,...,...
2623,White Dragon,Wrath
2624,White Dragon,Zeus
2625,Wonder Woman,Wrath
2626,Wonder Woman,Zeus


In [10]:
# Función para calcular la puntuación total de un héroe
def calcular_puntuacion(heroe, df):
    stats = df[df['heroe'] == heroe][['Intelligence', 'Strength', 'Speed', 'Durability', 'Power', 'Combat']].values[0]
    return sum(stats)

# Crear una nueva columna para los resultados
resultados = []

for index, row in enfrentamientos_df.iterrows():
    heroe_1 = row['Heroe_1']
    heroe_2 = row['Heroe_2']
    
    puntuacion_1 = calcular_puntuacion(heroe_1, df_DC)
    puntuacion_2 = calcular_puntuacion(heroe_2, df_DC)
    
    if puntuacion_1 > puntuacion_2:
        resultado = 1
    elif puntuacion_1 < puntuacion_2:
        resultado = 2
    else:
        resultado = 0
    
    resultados.append(resultado)

# Añadir la columna de resultados al dataframe de enfrentamientos
enfrentamientos_df['Resultado'] = resultados

# # Guardar el dataframe actualizado en un archivo CSV
# output_file_path_updated = '/mnt/data/enfrentamientos_heroes_dc_con_resultados.csv'
# enfrentamientos_df.to_csv(output_file_path_updated, index=False)

# output_file_path_updated


In [11]:
enfrentamientos_df

,Heroe_1,Heroe_2,Resultado
0,Aquaman,Ares,2
1,Aquaman,Atom Smasher,1
2,Aquaman,Batman,1
3,Aquaman,Black Adam,2
4,Aquaman,Black Canary,1
...,...,...,...
2623,White Dragon,Wrath,2
2624,White Dragon,Zeus,2
2625,Wonder Woman,Wrath,1
2626,Wonder Woman,Zeus,1


In [12]:

batallas_resultados = pd.read_csv('./data/enfrentamientos_heroes_dc_con_resultados_narrativos.csv')

In [13]:
batallas_resultados.head()

,Heroe_1,Heroe_2,Resultado_x,Resultado_y,Resultado
0,Aquaman,Ares,2,2.0,2.0
1,Aquaman,Atom Smasher,1,1.0,1.0
2,Aquaman,Batman,1,1.0,2.0
3,Aquaman,Black Adam,2,2.0,2.0
4,Aquaman,Black Canary,1,1.0,1.0


In [14]:
batallas_resultados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2628 entries, 0 to 2627
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Heroe_1      2628 non-null   object 
 1   Heroe_2      2628 non-null   object 
 2   Resultado_x  2628 non-null   int64  
 3   Resultado_y  5 non-null      float64
 4   Resultado    5 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 102.8+ KB


BATALLAS DC COMPLETO